In [24]:
def scale_in_silico_fc(functional_fc, in_silico_fc):
    # Determine true label
    true_label = 1 if functional_fc > 1 else (0 if functional_fc < 1 else 0.5)
    
    # Initialize prediction score
    initial_prediction = 0.5
    
    # Determine scaling based on functional fold change
    if functional_fc > 1:  # Positive fold change
        scaled_prediction = initial_prediction + ((in_silico_fc-1)/(functional_fc-1)) * 0.5  # Scale positively
    elif functional_fc < 1:  # Positive fold change
        scaled_prediction = initial_prediction - ((in_silico_fc+1)/(functional_fc+1)) * 0.5  # Scale positively
    else:
        scaled_prediction = initial_prediction
        
    # Clip predictions to stay within [0, 1]
    scaled_prediction = max(0, min(1, scaled_prediction))
    
    return true_label, scaled_prediction

# Example usage
functional_fc = -2  # 2-fold increase in functional assay
in_silico_fc = -1.1  # 1-fold increase in in silico model

true_label, scaled_prediction = scale_in_silico_fc(functional_fc, in_silico_fc)

print(f"True Label: {true_label}, Scaled Prediction: {scaled_prediction}")

True Label: 0, Scaled Prediction: 0.44999999999999996


In [1]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import celloracle as co
import anndata as ad
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams["savefig.dpi"] = 300
plt.rcParams["figure.figsize"] = [6, 4.5]

In [13]:
base_GRN = co.data.load_mouse_scATAC_atlas_base_GRN()


In [14]:
oracle.import_TF_data(TF_info_matrix=base_GRN)


TF dict already exists. The old TF dict data was deleted. 



In [2]:
oracle = co.load_hdf5("jinlab/deg_grn/ctxobj.Tbr1_1.CR.peturbseq.subclass.celloracle.oracle")

In [10]:
oracle.adata.obs["predicted.subclass"].value_counts()

L6 CT CTX        3188
CR               1324
L6 IT CTX         877
L5 NP CTX         876
L5 IT CTX         651
L5 IT TPE-ENT     645
L5 PT CTX         525
Sst               468
Pvalb             415
L2/3 IT CTX-1     406
L2 IT ENTl        340
L6b/CT ENT        306
L2/3 IT CTX-2     296
L4/5 IT CTX       252
L6b CTX           249
NP SUB            182
Car3              122
L2/3 IT ENTl       79
L6 IT ENTl         79
Sncg               42
L2 IT RHP          30
Vip                26
Lamp5              19
L3 IT ENT          18
L2/3 IT PPP        12
CT SUB              9
Oligo               9
NP PPP              7
CA2                 6
CA3                 5
Sst Chodl           5
Astro               1
SMC-Peri            1
L5 PPP              1
Name: predicted.subclass, dtype: int64

In [12]:
astro_row = oracle.adata[oracle.adata.obs["predicted.subclass"] == "Astro"]
astro_row.to_df()

,Gm5577,Asns,Tmem39a,Mocs3,Cpsf4,Fbxo11,Naa25,Dgat1,Gm37401,Eml5,...,Ntng1,Tmem91,Tmem196,Vldlr,Cmklr1,Efhd1,Fabp3,Kndc1,Gabra3,Tbr1
Ch4_TTGGAACAGCCACCTG-1,0.0,1.272515,0.0,0.414731,0.0,0.414731,0.0,0.414731,0.0,0.0,...,0.414731,0.0,0.414731,0.0,0.0,0.0,2.467526,0.0,0.0,0.707014


In [18]:
genes_in_scRNA = set(oracle.adata.var_names)
tf_genes_in_dict = set(oracle.TFdict.keys())
overlapping_genes = genes_in_scRNA.intersection(tf_genes_in_dict)

print(f"Number of overlapping genes: {len(overlapping_genes)}")


Number of overlapping genes: 2518


In [20]:
# Assuming your list of var names is stored in a variable called 'var_names'
var_names = oracle.adata.var_names.tolist()  # Convert to a list if it's not already

# Find duplicates
duplicates = [name for name in set(var_names) if var_names.count(name) > 1]

# Output the result
if duplicates:
    print(f"Found duplicates: {duplicates}")
else:
    print("No duplicates found.")

Found duplicates: ['Tbr1']


In [19]:
%%time
# Calculate GRN for each population in "louvain_annot" clustering unit.
# This step may take some time.(~30 minutes)
links = oracle.get_links(cluster_name_for_GRN_unit="predicted.subclass", alpha=10,
                         verbose_level=10, test_mode=True, bagging_number=10)

  0%|          | 0/34 [00:00<?, ?it/s]

Inferring GRN for Astro...


  0%|          | 0/2518 [00:00<?, ?it/s]

IndexError: index 79 is out of bounds for axis 0 with size 79

input file name: /gpfs/home/asun/jinlab/ctxobj.rds


Loading required package: SeuratObject
Loading required package: sp

Execution halted


KeyboardInterrupt: 

In [22]:
df = pd.read_csv(f"/gpfs/home/asun/jinlab/tsv/edgeR_LRT_with_sva.Tbr1_1.CR.tsv", sep='\t')
df_sorted = df.iloc[df['logFC'].abs().argsort()]
df_sorted.set_index("Unnamed: 0")
rows_to_add = df_sorted.loc[df_sorted["Unnamed: 0"] == "Tbr1"]
DEG = df_sorted.tail(3000)
DEG = pd.concat([DEG, rows_to_add], ignore_index=True)
DEG.set_index("Unnamed: 0")
deg_list = DEG['Unnamed: 0'].unique().tolist()


In [23]:
deg_list

['Gm5577',
 'Asns',
 'Tmem39a',
 'Mocs3',
 'Cpsf4',
 'Fbxo11',
 'Naa25',
 'Dgat1',
 'Gm37401',
 'Eml5',
 'Zkscan6',
 'Arid5a',
 'Nat10',
 'Selenon',
 'Kcnd3',
 'Ankrd49',
 'Pkig',
 'Cadps',
 'Ankrd6',
 'Bscl2',
 'Rps27rt',
 'Lrrc24',
 'Enah',
 'Exosc1',
 'Trmu',
 'Slc6a17',
 'Zfp322a',
 'Tfdp1',
 'Med16',
 'Stradb',
 'Mid2',
 'Ttc9c',
 'Hlcs',
 'Gtpbp1',
 'AW554918',
 'Fam13c',
 '0610010F05Rik',
 'Cxxc1',
 'Mxd1',
 '2300009A05Rik',
 'Glrx',
 'Nxn',
 'Pcyox1l',
 'Nxph3',
 'Vasp',
 'Skil',
 'Pea15a',
 'Pkn2',
 'Borcs5',
 'Zfp846',
 'Tsc22d3',
 'Zfp422',
 'Lratd1',
 'Edc4',
 'Smim27',
 'Snap29',
 'Slc25a51',
 'Smim8',
 'Socs6',
 'Inip',
 'Prxl2c',
 'Hadha',
 'Kat6b',
 'Mrps18b',
 'Ptgds',
 'Avl9',
 'Tbr1',
 'Zfp532',
 'Prickle2',
 'Gpr173',
 'Bicral',
 'Unc13b',
 'Lrch2',
 'Txnrd2',
 'Ccar2',
 'Atp1b3',
 'Hdac10',
 'Tcaim',
 'Ube2k',
 'Kif17',
 'Panx2',
 'Cbx7',
 'Eno1',
 'Mblac2',
 'Qrsl1',
 'Lyrm1',
 'Pcca',
 'Atg3',
 'Tpm4',
 'A830082N09Rik',
 'Pisd',
 'Ado',
 'Pdik1l',
 'Riok2',
 'Rrp